Importing the useful libraries

In [206]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import pbs
from pickle_vocab import *
from cooc import *

## Loading the data

In [207]:
def load_test_data(data_path):
    """Loads data and returns ids (event ids) and X (feature vector)"""
    
    f = open(data_path)
    x = f.readlines()
    tweets = []
    ids = []
    for str in x :
        id, tweet = str.split(',',1)
        tweets.append(tweet)
        ids.append(id)

    x = pd.DataFrame(tweets,ids,columns=['Tweets'])
    
    return x

test = load_test_data('Datasets/test_data.txt')
test.head()

,Tweets
1,sea doo pro sea scooter ( sports with the port...
2,<user> shucks well i work all week so now i ca...
3,i cant stay away from bug thats my baby\n
4,<user> no ma'am ! ! ! lol im perfectly fine an...
5,"whenever i fall asleep watching the tv , i alw..."


In [208]:
def load_train_data(data_path):
    """Loads data and returns ids (event ids) and X (feature vector)"""
    
    f = open(data_path)
    x = f.readlines()
    x = pd.DataFrame(x,columns=['Tweets'])
    
    return x

In [209]:
pos = load_train_data('Datasets/train_pos.txt')
pos.head()

,Tweets
0,<user> i dunno justin read my mention or not ....
1,"because your logic is so dumb , i won't even c..."
2,""" <user> just put casper in a box ! "" looved t..."
3,<user> <user> thanks sir > > don't trip lil ma...
4,visiting my brother tmr is the bestest birthda...


In [210]:
neg = load_train_data('Datasets/train_neg.txt')
neg.head()

,Tweets
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...
1,glad i dot have taks tomorrow ! ! #thankful #s...
2,1-3 vs celtics in the regular season = were fu...
3,<user> i could actually kill that girl i'm so ...
4,<user> <user> <user> i find that very hard to ...


In [211]:
pos["y"]=1
pos.head()

,Tweets,y
0,<user> i dunno justin read my mention or not ....,1
1,"because your logic is so dumb , i won't even c...",1
2,""" <user> just put casper in a box ! "" looved t...",1
3,<user> <user> thanks sir > > don't trip lil ma...,1
4,visiting my brother tmr is the bestest birthda...,1


In [212]:
neg["y"]=-1
neg.head()

,Tweets,y
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...,-1
1,glad i dot have taks tomorrow ! ! #thankful #s...,-1
2,1-3 vs celtics in the regular season = were fu...,-1
3,<user> i could actually kill that girl i'm so ...,-1
4,<user> <user> <user> i find that very hard to ...,-1


In [213]:
train = pd.concat([pos,neg])
train.reset_index(drop=True,inplace=True)
train.head()

,Tweets,y
0,<user> i dunno justin read my mention or not ....,1
1,"because your logic is so dumb , i won't even c...",1
2,""" <user> just put casper in a box ! "" looved t...",1
3,<user> <user> thanks sir > > don't trip lil ma...,1
4,visiting my brother tmr is the bestest birthda...,1


In [214]:
print("following rows get NaN weights")
train.loc[[29049, 52357, 56151, 68379, 69973, 114433, 133011, 175789, 190542]]

following rows get NaN weights


,Tweets,y
29049,heyheyheyhey\n,1
52357,#ireland #ireland #ireland\n,1
56151,gilipollas #everywhere\n,1
68379,#absolutely beastingit\n,1
69973,#haveagooddaytomorrow #goodnightworld\n,1
114433,#killingmesoftly\n,-1
133011,catss everywheree\n,-1
175789,straks nederland toets\n,-1
190542,ziekjes thuis\n,-1


In [215]:
test.loc[['4590']]

,Tweets
4590,#thatsjustmethough\n


In [216]:
with open('vocab.pkl','rb') as f:
    vocab = pickle.load(f)
vocab

{'<user>': 0,
 '!': 1,
 'i': 2,
 'the': 3,
 '.': 4,
 ',': 5,
 'to': 6,
 'you': 7,
 '(': 8,
 '<url>': 9,
 'a': 10,
 '...': 11,
 'and': 12,
 'my': 13,
 'me': 14,
 'of': 15,
 '?': 16,
 'is': 17,
 'for': 18,
 'in': 19,
 'it': 20,
 '"': 21,
 'this': 22,
 'so': 23,
 '-': 24,
 'with': 25,
 'on': 26,
 'that': 27,
 ')': 28,
 'be': 29,
 "i'm": 30,
 'have': 31,
 ':': 32,
 'but': 33,
 'just': 34,
 'rt': 35,
 'love': 36,
 'your': 37,
 'all': 38,
 'not': 39,
 'was': 40,
 'at': 41,
 'are': 42,
 '..': 43,
 'like': 44,
 '/': 45,
 'get': 46,
 'up': 47,
 'frame': 48,
 '&': 49,
 'lol': 50,
 'know': 51,
 'good': 52,
 'do': 53,
 'u': 54,
 'now': 55,
 'when': 56,
 'one': 57,
 'if': 58,
 'we': 59,
 'follow': 60,
 'no': 61,
 'can': 62,
 'what': 63,
 'go': 64,
 "don't": 65,
 'out': 66,
 'x': 67,
 'will': 68,
 'day': 69,
 "'": 70,
 'please': 71,
 'from': 72,
 'see': 73,
 'too': 74,
 'want': 75,
 'there': 76,
 'back': 77,
 "it's": 78,
 'today': 79,
 'about': 80,
 'really': 81,
 'how': 82,
 'got': 83,
 'thanks': 8

In [217]:
max(vocab.values())

21160

In [8]:
#!/usr/bin/env python3
from scipy.sparse import *
import numpy as np
import pickle


def cooc_full():
    with open('vocab_full.pkl', 'rb') as f:
        vocab = pickle.load(f)
    vocab_size = len(vocab)

    data, row, col = [], [], []
    counter = 1
    for fn in ['Datasets/train_pos_full.txt', 'Datasets/train_neg_full.txt']:
        with open(fn) as f:
            for line in f:
                tokens = [vocab.get(t, -1) for t in line.strip().split()]
                tokens = [t for t in tokens if t >= 0]
                for t in tokens:
                    for t2 in tokens:
                        data.append(1)
                        row.append(t)
                        col.append(t2)

                if counter % 10000 == 0:
                    print(counter)
                counter += 1
    cooc = coo_matrix((data, (row, col)))
    print("summing duplicates (this can take a while)")
    cooc.sum_duplicates()
    with open('cooc_full.pkl', 'wb') as f:
        pickle.dump(cooc, f, pickle.HIGHEST_PROTOCOL)
        
        
cooc_full = cooc_full()

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe2 in position 0: ordinal not in range(128)

In [218]:
with open('cooc.pkl','rb') as f:
    cooc = pickle.load(f)
cooc

<21160x21160 sparse matrix of type '<class 'numpy.int32'>'
	with 3184638 stored elements in COOrdinate format>

In [219]:
cooc.data

array([273434,  97020,  57576, ...,      2,      2,      8])

In [220]:
cooc.row

array([    0,     1,     2, ..., 12840, 14651, 21159])

In [221]:
cooc.col

array([    0,     0,     0, ..., 21159, 21159, 21159])

In [222]:
print(cooc)

  (0, 0)	273434
  (1, 0)	97020
  (2, 0)	57576
  (3, 0)	38886
  (4, 0)	47366
  (5, 0)	41118
  (6, 0)	41428
  (7, 0)	58492
  (8, 0)	4744
  (9, 0)	10364
  (10, 0)	31758
  (11, 0)	12300
  (12, 0)	27856
  (13, 0)	21426
  (14, 0)	22728
  (15, 0)	14316
  (16, 0)	29786
  (17, 0)	15902
  (18, 0)	20674
  (19, 0)	14742
  (20, 0)	21172
  (21, 0)	19938
  (22, 0)	8938
  (23, 0)	12654
  (24, 0)	6002
  :	:
  (208, 21159)	4
  (239, 21159)	2
  (250, 21159)	2
  (273, 21159)	2
  (314, 21159)	6
  (349, 21159)	2
  (444, 21159)	2
  (676, 21159)	2
  (693, 21159)	2
  (883, 21159)	2
  (1048, 21159)	2
  (1050, 21159)	2
  (1388, 21159)	2
  (1415, 21159)	2
  (2291, 21159)	2
  (2915, 21159)	2
  (3359, 21159)	2
  (6055, 21159)	2
  (7507, 21159)	2
  (7875, 21159)	2
  (7879, 21159)	2
  (9534, 21159)	2
  (12840, 21159)	2
  (14651, 21159)	2
  (21159, 21159)	8


In [223]:
cooc.toarray()

array([[273434,  97020,  57576, ...,      0,     10,      4],
       [ 97020, 255078,  47124, ...,      0,      2,      0],
       [ 57576,  47124, 110506, ...,      0,      0,      4],
       ...,
       [     0,      0,      0, ...,      0,      0,      0],
       [    10,      2,      0, ...,      0,     10,      0],
       [     4,      0,      4, ...,      0,      0,      8]])

In [224]:
cooc.toarray().shape

(21160, 21160)

In [225]:
we = np.load("embeddings.npy")
we

array([[ 0.08293404,  0.01490157,  0.00622929, ..., -0.16190348,
        -0.1846269 , -0.18189624],
       [ 0.12380148, -0.0421271 , -0.01440238, ..., -0.30058427,
        -0.22970339, -0.25452988],
       [ 0.09969781,  0.00634767, -0.05967447, ..., -0.30113178,
        -0.29646062, -0.36644036],
       ...,
       [-0.03857365, -2.05255734, -0.04303483, ..., -0.78177959,
         0.37153678, -0.62341192],
       [ 2.09123134, -1.04694662, -0.92195585, ...,  0.26456563,
         0.18419511, -0.29720033],
       [ 1.03783875, -0.6643759 , -0.19507841, ...,  0.31626424,
        -3.13279291,  1.45051465]])

In [226]:
we.shape

(21160, 20)

In [227]:
def representation(tweet):
    acc = np.array(0)
    n_ignored_word = 0
    for word in tweet.split():
        if word not in vocab.keys():
            n_ignored_word += 1
        else:
            try:
                acc = np.add(acc,we[vocab[word]])
            except: 
                print("problem with " + word) #last word from vocab is missing in cooc
                n_ignored_word += 1
    n = len(tweet.split()) - n_ignored_word
    acc = acc/n
    return(acc)

In [228]:
train["w"] = train["Tweets"].apply(lambda x: representation(x))

problem with #10factsaboutme
problem with #10factsaboutme
problem with #10factsaboutme
problem with #10factsaboutme
problem with #10factsaboutme


In [229]:
col = ["w" + str(k) for k in range(20)]
col

['w0',
 'w1',
 'w2',
 'w3',
 'w4',
 'w5',
 'w6',
 'w7',
 'w8',
 'w9',
 'w10',
 'w11',
 'w12',
 'w13',
 'w14',
 'w15',
 'w16',
 'w17',
 'w18',
 'w19']

In [230]:
train[col] = train["w"].apply(pd.Series)
train.drop("w",axis=1,inplace=True)
train.drop("Tweets",axis=1,inplace=True)

In [231]:
train

,y,w0,w1,w2,w3,w4,w5,w6,w7,w8,...,w10,w11,w12,w13,w14,w15,w16,w17,w18,w19
0,1,0.193815,0.038084,0.040004,-0.668191,0.548065,-0.373069,0.370133,0.348918,0.419950,...,-0.300119,0.397517,0.566418,0.117509,0.803911,0.124804,0.664562,-0.236155,-0.140885,-0.315508
1,1,0.164580,-0.173796,0.008294,-0.758810,0.600187,-0.400949,0.140147,0.408942,0.340775,...,-0.098460,0.442516,0.533248,0.247309,0.549468,0.173431,0.633311,-0.190100,-0.074907,-0.354327
2,1,0.174736,-0.009662,-0.141506,-0.549435,0.538021,-0.390385,0.205312,0.320152,0.582864,...,-0.017894,0.298703,0.484783,0.156315,0.592756,0.358672,0.746317,-0.339098,-0.251291,-0.313407
3,1,-0.053728,-0.128535,0.083332,-0.790976,0.578776,-0.068607,0.170223,0.189749,0.468706,...,-0.177187,0.137006,0.338732,0.324995,0.558550,0.114529,0.642133,-0.077130,-0.280407,-0.424682
4,1,-0.000336,-0.311349,0.063379,-0.652141,0.242091,-0.023745,0.237209,0.254238,0.225467,...,-0.140884,0.474569,0.570961,0.346219,0.603160,0.214515,0.475464,-0.274569,-0.285542,-0.216674
5,1,0.026594,-0.006907,-0.100526,-0.749362,0.524098,-0.346801,0.243778,0.348224,0.376137,...,-0.173890,0.358738,0.535558,0.141857,0.736740,0.082623,0.739355,-0.259015,-0.265909,-0.379111
6,1,0.090643,-0.048549,-0.010043,-0.657629,0.385867,-0.272697,0.067982,0.290498,0.352461,...,-0.208982,0.462297,0.350906,0.373653,0.659256,0.136557,0.673275,-0.289193,-0.106955,-0.293110
7,1,0.105990,0.324250,-0.020413,-0.368366,0.482513,-0.298971,-0.031015,0.283898,0.488445,...,-0.085626,0.093791,0.237290,0.314822,0.559136,0.289033,0.506103,-0.281387,-0.219288,-0.260018
8,1,0.057533,-0.066158,0.033396,-0.654281,0.382003,-0.349268,0.261607,0.506702,0.347005,...,0.090888,0.428261,0.358533,0.156962,0.712901,0.053821,0.711513,-0.304527,-0.313385,-0.482374
9,1,0.195404,0.123427,-0.582616,-0.908895,0.608877,-0.441935,0.092457,0.419204,0.437667,...,-0.341651,0.301421,-0.005913,-0.119228,0.481244,0.218687,0.595256,-0.038230,-0.192498,-0.554049


In [232]:
test.head()

,Tweets
1,sea doo pro sea scooter ( sports with the port...
2,<user> shucks well i work all week so now i ca...
3,i cant stay away from bug thats my baby\n
4,<user> no ma'am ! ! ! lol im perfectly fine an...
5,"whenever i fall asleep watching the tv , i alw..."


In [233]:
test["w"] = test["Tweets"].apply(lambda x: representation(x))
test[col] = test["w"].apply(pd.Series)
test.drop("w",axis=1,inplace=True)
test.drop("Tweets",axis=1,inplace=True)

problem with #10factsaboutme


In [234]:
test

,w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,w14,w15,w16,w17,w18,w19
1,0.259533,0.059221,-0.450866,-0.689856,0.480041,-0.267784,0.051785,0.359576,0.494638,0.120045,-0.185360,0.557172,0.507819,0.107620,0.542991,-0.045309,0.342527,-0.196790,-0.439141,-0.308582
2,0.150485,-0.073187,-0.203847,-0.686830,0.483774,-0.184477,0.248559,0.150328,0.346024,-0.007975,-0.261741,0.296996,0.547765,-0.002013,0.708832,0.237233,0.631884,-0.254344,-0.244305,-0.268326
3,0.109122,-0.055159,-0.075084,-0.602511,0.323006,-0.402221,0.205848,0.491140,0.413059,-0.107106,-0.222141,0.342126,0.476687,0.256765,0.619973,0.047600,0.717036,-0.269757,-0.305203,-0.229819
4,0.200103,0.153898,-0.171113,-0.550022,0.206411,-0.279831,0.057420,0.361296,0.258558,-0.019996,-0.122725,0.322543,0.276396,0.134913,0.817255,-0.026374,0.644372,-0.504629,-0.206761,-0.277321
5,0.271728,-0.070988,-0.006069,-0.511087,0.414814,-0.379588,0.132693,0.283727,0.483064,0.062769,-0.295117,0.355119,0.554627,0.214441,0.567460,0.126346,0.606131,-0.426146,-0.315099,-0.230338
6,0.177140,-0.064049,-0.235508,-0.671921,0.468047,-0.351670,0.123075,0.417831,0.360304,-0.032732,-0.116584,0.438575,0.328540,0.272162,0.622874,0.154174,0.768459,-0.156917,-0.233959,-0.193053
7,0.050540,0.023630,-0.318031,-0.700890,0.656585,-0.121047,0.151539,0.208249,0.376164,-0.042006,-0.146891,0.424010,0.646687,0.260476,0.660293,-0.085088,0.691258,-0.264102,-0.326783,-0.194410
8,0.079734,-0.147615,-0.081583,-0.771016,0.457424,-0.368448,0.115835,0.293994,0.370616,-0.259839,-0.174606,0.229767,0.545067,0.234551,0.754708,-0.037113,0.669069,-0.298692,-0.250571,-0.286325
9,0.069434,-0.049755,-0.068777,-0.770877,0.539732,-0.381339,0.183634,0.351502,0.487904,-0.127915,-0.218412,0.373151,0.487160,0.101285,0.847474,0.132254,0.741720,-0.297788,-0.352823,-0.340416
10,0.088795,-0.036685,-0.089279,-0.693793,0.477863,-0.204369,0.208715,0.417147,0.443159,-0.089004,-0.273531,0.289795,0.490565,0.098937,0.548331,0.185409,0.798275,-0.205321,-0.206249,-0.303574


In [235]:
train[train.isnull().any(axis=1)].index

Int64Index([29049, 52357, 56151, 68379, 69973, 114433, 133011, 175789, 190542], dtype='int64')

In [236]:
train.dropna(inplace=True)

In [237]:
test[test.isnull().any(axis=1)].index

Index(['4590'], dtype='object')

In [238]:
test.dropna(inplace=True)

In [239]:
X = train.drop("y", axis=1)
y = train["y"]

In [240]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier()
clf.fit(X, y) 
test["p"]=clf.predict(test)

You should first create a folder named **Submissions**

In [ ]:
test["p"].to_csv("Submissions/submission.csv",header=False)

## Data Analysis

## Word enbeddings matrix

### Generation

### Visualization

## Building the text classifier

### Training texts features

### Linear classifiers

### Predictions

## Output creation and visualization

### Creating the submission file

### Visualization of the results